#### This lab aims to train and test an object detection model using transfer learning capable of detecting 4 classes: bottle, can, cup, nocup

#### This lab was inspired by this course. 
#### Course Link : https://www.youtube.com/watch?v=yqkISICHH-U&t=18602s&ab_channel=NicholasRenotte

#### The pretrained models used in this lab are part of the TensorFlow detection Zoo. The TensorFlow detection Zoo is a collection of pretrained object detection models trained on the COCO 2017 dataset.
#### GitHub link : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

#### Within this lab, we will be testing diffrent pretrained models from the TensorFlow detection Zoo and some of the well known models like YOLO. The main goal is to pick up the pretrained model with the heighest accuracy.

#### Note : As the data collection phase is not completed yet, and as training more than one model can be computationally expensive, this lab will focus on training and testing only one pretrained model to demonstrate the hole process of training and testing an object detecting model. Once the data collection phase is completed, then we will proceede with the other models. 
#### The pretrained model implemented within this lab will be the centernet_hg104_512x512 model from the TensorFlow detection Zoo.

In [2]:
import os

In [2]:
# Downloading the pretrained centernet_hg104_512x512 object detection model
!wget download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz

--2023-05-17 17:36:06--  http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.125.128, 2607:f8b0:4001:c2f::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.125.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1426099846 (1.3G) [application/x-tar]
Saving to: ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’

centernet_hg104_512 100%[===================>]   1.33G   248MB/s    in 5.6s    

2023-05-17 17:36:12 (243 MB/s) - ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’ saved [1426099846/1426099846]



In [3]:
# Extracting the .tar.gz file of the pretrained_model 
!tar -zxvf centernet_hg104_512x512_coco17_tpu-8.tar.gz

centernet_hg104_512x512_coco17_tpu-8/
centernet_hg104_512x512_coco17_tpu-8/checkpoint/
centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
centernet_hg104_512x512_coco17_tpu-8/checkpoint/checkpoint
centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.index
centernet_hg104_512x512_coco17_tpu-8/pipeline.config
centernet_hg104_512x512_coco17_tpu-8/saved_model/
centernet_hg104_512x512_coco17_tpu-8/saved_model/saved_model.pb
centernet_hg104_512x512_coco17_tpu-8/saved_model/assets/
centernet_hg104_512x512_coco17_tpu-8/saved_model/variables/
centernet_hg104_512x512_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
centernet_hg104_512x512_coco17_tpu-8/saved_model/variables/variables.index


In [4]:
# Removing the .tar.gz file of the pretrained_model (no need for the compressed file of the model)
!rm centernet_hg104_512x512_coco17_tpu-8.tar.gz

#### Training an object detection model consists of feeding the model with a bunch of annotated (labeled) images to learn from. Annotating(labeling) an image refers to drawing a bounding box around the object we want to detect and specifying the label(class) of that object. In our case, we have 4 classes(labels) that we want our model to detect(bottle, can, cup, nocup).
#### To annotate (label) collected images, we used a tool named labelImg. Annotating (labeling) the images will generate a .xml file for each annotated(labeled) image.
#### You can learn more about labeling images and the labelImg tool that we used to annotate images from the course (course link above). you can skip to the  Labelling Images for Object Detection using LabelImg part at 1:04:11.

In [3]:
# Importing the labeled(annotated) images that will be used to train and test the object detection model
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
# Just Copying the dataset compressed file from the drive folder to the main folder (/content/)
# The dataset contains the train and test datasets that contains respectively the labeled(annotated) images for training and testing the object detection model
!cp drive/MyDrive/dataset_for_object_detection_model.zip .

In [4]:
!cp drive/MyDrive/object_detection_model.zip .

In [5]:
# Unzipping the dataset compressed file
!unzip dataset_for_object_detection_model.zip

Archive:  object_detection_model.zip
   creating: object_detection_model/
  inflating: object_detection_model/pipeline.config  
   creating: object_detection_model/saved_model/
   creating: object_detection_model/saved_model/variables/
  inflating: object_detection_model/saved_model/variables/variables.index  
  inflating: object_detection_model/saved_model/variables/variables.data-00000-of-00001  
   creating: object_detection_model/saved_model/assets/
  inflating: object_detection_model/saved_model/saved_model.pb  
 extracting: object_detection_model/saved_model/fingerprint.pb  
   creating: object_detection_model/checkpoint/
  inflating: object_detection_model/checkpoint/ckpt-0.data-00000-of-00001  
  inflating: object_detection_model/checkpoint/ckpt-0.index  
  inflating: object_detection_model/checkpoint/checkpoint  
  inflating: object_detection_model/label_map.pbtxt  


In [6]:
# Remove the  compressed dataset file(no need for the compressed files)
!rm dataset_for_object_detection_model.zip

### Generating TF Records :
#### TF Records are binary file format for storing data.
#### Until now we possess a bunch of .jpg images along with .xml files generated from annotating these images. We said that training an object detection model consists of feeding the model with a bunch of annotated (labeled) images to learn from. But until this moment, we still have the images and their annotations as 2 seprate files. And here comes the TF Records as a combination of the images and their annotations in one single data structure.
#### + Using TF Records speed up training for your custom object detection model.

#### Before we can generate these TF Records, we need to create a label_map file in which we map each label (bottle, cup, nocup, can) to an integer (1, 2, 3, 4). This label_map file will be used to generate the TF Records, and then the TF Records will be used to train and test the model.

In [9]:
# Let's start with creating the label_map.pbtxt file
!touch label_map.pbtxt

In [ ]:
# Mapping each label to an integer
# We can do this manually : just open the label_map.pbtxt file manually and add what's inside the brackets:
"""
item {
	name:'bottle'
	id:1
}

item {
	name:'cup'
	id:2
}

item {
	name:'nocup'
	id:3
}

item {
	name:'can'
	id:4
}
"""

In [7]:
# Cloning the tenorflow model garden repository from GitHub. This repo contains 
# a bunch of tools and modeling solutions that will be used to genrate the TF 
# Records and will be used later to train and test the model
!git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 85559, done.
remote: Counting objects: 100% (1158/1158), done.
remote: Compressing objects: 100% (453/453), done.
remote: Total 85559 (delta 713), reused 1110 (delta 699), pack-reused 84401
Receiving objects: 100% (85559/85559), 598.65 MiB | 34.94 MiB/s, done.
Resolving deltas: 100% (61220/61220), done.


In [8]:
# installing the object_detection dependency from the cloned repository
!cd models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 22.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Prep

In [12]:
# Cloning a github repository where we can find a python script that can generate these TF Records
!git clone https://github.com/nicknochnack/GenerateTFRecord

Cloning into 'GenerateTFRecord'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 2.67 KiB | 2.67 MiB/s, done.


In [13]:
# Genrate the train.record file that will be used to train the model
!python GenerateTFRecord/generate_tfrecord.py -x dataset_for_object_detection_model/train/ -l label_map.pbtxt -o train.record

Successfully created the TFRecord file: train.record


In [14]:
# Genrate the test.record file that will be used to test the model
!python GenerateTFRecord/generate_tfrecord.py -x dataset_for_object_detection_model/test/ -l label_map.pbtxt -o test.record

Successfully created the TFRecord file: test.record


In [15]:
# For a better organization and consistency of our work, we will create a new directory named annotations
# that will hold the train.record, test.record and the label_map.pbtxt files.
!mkdir annotations
!mv train.record annotations/
!mv test.record annotations/
!mv label_map.pbtxt annotations/

In [16]:
# Now we will create a new directory named my_custom_model that will contain
# our custom trained object detection model later on.
!mkdir my_custom_model

In [17]:
# Now we will copy the pipeline.config file from the centernet pretrained model directory 
# to our my_custom_model directory.
# The pipeline.config file will be used to configure the parameters and initial settings needed to train and test the model
# (path to the train.record/test.record files, path to pretrained_model, etc...) 
!cp centernet_hg104_512x512_coco17_tpu-8/pipeline.config my_custom_model

In [ ]:
# Now we need to finish setting up the the configuration in the pipeline.config file.
# We can do this by manually opening the pipeline.config file in the my_custom_model directory  
# and start configuring the parameters.
# Scroll down slowly through the pipeline.config file and change the parameters as shown.
# The parameters to configure are:
"""
num_classes: 4
image_resizer {
  fixed_shape_resizer {
    height: 512
    width: 512
  }
}
train_config{
  batch_size : 8
  }
fine_tune_checkpoint: "/content/centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0"
train_input_reader {
  label_map_path : "/content/annotations/label_map.pbtxt"
  input_path : "/content/annotations/train.record"
}
eval_input_reader {
  label_map_path: "/content/annotations/label_map.pbtxt"
  input_path: "/content/annotations/test.record"
}
"""
# We can still configure more of the parameters in the pipeline.config file like the optimizer, num_steps etc...
# But for now, we will just focus on the most important parameters and let the optimisation in the finetuning phase later on.

### Training the model


In [20]:
!python models/research/object_detection/model_main_tf2.py --model_dir=my_custom_model --pipeline_config_path=my_custom_model/pipeline.config --num_train_steps=5000

2023-05-17 18:49:59.512154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 18:50:00.981094: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-05-17 18:50:04.932611: I tensorflow/compil

### Testing the model

In [21]:
!python models/research/object_detection/model_main_tf2.py --model_dir=my_custom_model --pipeline_config_path=my_custom_model/pipeline.config --checkpoint_dir=my_custom_model

Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 114, in <module>
    tf.compat.v1.app.run()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/platform/app.py", line 36, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.10/dist-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.10/dist-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/content/models/research/object_detection/model_main_tf2.py", line 81, in main
    model_lib_v2.eval_continuously(
  File "/usr/local/lib/python3.10/dist-packages/object_detection/model_lib_v2.py", line 1135, in eval_continuously
    for latest_checkpoint in tf.train.checkpoints_iterator(
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/training/checkpoint_utils.py", line 244, in checkpoints_iterator
KeyboardInterrupt
^C


In [ ]:
### Average Precision of 76% is not bad for a start.

### Exporting the model

In [22]:
# Creating a directory named object_detection_model that will contain our trained model
!mkdir object_detection_model

In [23]:
# freezing the model
!python models/research/object_detection/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=my_custom_model/pipeline.config --trained_checkpoint_dir=my_custom_model --output_directory=object_detection_model

2023-05-17 21:09:45.005761: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 21:09:46.617748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-05-17 21:09:49.464886: I tensorflow/compil

### Convert our model to TFLite 
#### TFLite is a lighter version of our model which can be deployed in an embedded system such as a raspberry pi

In [10]:
!python models/research/object_detection/export_tflite_graph_tf2.py  --pipeline_config_path=object_detection_model/pipeline.config --trained_checkpoint_dir=object_detection_model/checkpoint checkpoint --output_directory=object_detection_model_tflite

2023-05-17 21:31:00.071386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 21:31:01.768902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-05-17 21:31:05.397647: I tensorflow/compil

In [11]:
!tflite_convert --saved_model_dir=object_detection_model_tflite/saved_model --output_file=object_detection_model_tflite/saved_model/object_detection_model.tflite --input_shapes=1,512,512,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops

2023-05-17 21:35:26.842859: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 21:35:28.132473: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-17 21:35:29.932694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-17 21:35:29.978903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

### Compressing and exporting our models 



In [26]:
!cp /content/annotations/label_map.pbtxt object_detection_model

In [27]:
!zip -r object_detection_model.zip object_detection_model
!cp object_detection_model.zip drive/MyDrive

  adding: object_detection_model/ (stored 0%)
  adding: object_detection_model/pipeline.config (deflated 68%)
  adding: object_detection_model/saved_model/ (stored 0%)
  adding: object_detection_model/saved_model/variables/ (stored 0%)
  adding: object_detection_model/saved_model/variables/variables.index (deflated 80%)
  adding: object_detection_model/saved_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: object_detection_model/saved_model/assets/ (stored 0%)
  adding: object_detection_model/saved_model/saved_model.pb (deflated 95%)
  adding: object_detection_model/saved_model/fingerprint.pb (stored 0%)
  adding: object_detection_model/checkpoint/ (stored 0%)
  adding: object_detection_model/checkpoint/ckpt-0.data-00000-of-00001 (deflated 7%)
  adding: object_detection_model/checkpoint/ckpt-0.index (deflated 83%)
  adding: object_detection_model/checkpoint/checkpoint (deflated 41%)
  adding: object_detection_model/label_map.pbtxt (deflated 50%)


In [28]:
!cp /content/annotations/label_map.pbtxt object_detection_model_tflite

In [12]:
!zip -r object_detection_model_tflite.zip object_detection_model_tflite
!cp object_detection_model_tflite.zip drive/MyDrive

  adding: object_detection_model_tflite/ (stored 0%)
  adding: object_detection_model_tflite/saved_model/ (stored 0%)
  adding: object_detection_model_tflite/saved_model/variables/ (stored 0%)
  adding: object_detection_model_tflite/saved_model/variables/variables.index (deflated 80%)
  adding: object_detection_model_tflite/saved_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: object_detection_model_tflite/saved_model/assets/ (stored 0%)
  adding: object_detection_model_tflite/saved_model/object_detection_model.tflite (deflated 7%)
  adding: object_detection_model_tflite/saved_model/saved_model.pb (deflated 95%)
  adding: object_detection_model_tflite/saved_model/fingerprint.pb (stored 0%)
